# Video Frame Extractor
#### A simple video frame extractor that allows you to take out frames from videos to say train your machine learning application

##### To use this script, specify your input_folder_path, full_folder (whether you want to extract all videos from the specified folder or only a specific few (file_names)); your output_folder_path, and single_folder (whether you want everything to go into the same folder or be seperated under output_folder_path); Optional: subwindow coordinates: (x1, y1, x2, y2) to only take the frames out of a subwindow of the original video

In [1]:
# Input path information
input_folder_path = r"/app/Videos"
full_folder = True
file_names = ["Joker.mp4","Trashtalk.mp4"]

# Output path information
output_folder_path = r"/app/TestVideos"
single_folder = False

# Subwindow coordinates : (x1, y1, x2, y2)
# subwindow_coords = (100, 100, 400, 400) 
# If you want the full window size:
subwindow_coords = None

# Video file extensions
video_extensions = ['.mp4', '.avi', '.mov', '.mkv'] 


In [2]:
import cv2                                                      # Note, if missing import, do pip install opencv-python
import os

In [ ]:
# %pwd

In [3]:
# Check if a given file has the correct file extension
def is_valid_video_file(file_path):
    return any(file_path.lower().endswith(ext) for ext in video_extensions)

In [4]:
# Extracting frames from 1 video
def extract_frames(video_path, output_path, frame_name_prefix='frame', subwindow_coords = None):
    # 1. Prepare the output directory if it doesn't exist already
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    # 2. Open the video file
    vid_cap = cv2.VideoCapture(video_path)
    success, image = vid_cap.read()
    count = 0

    # 3. Loop through video frames and save them as images
    while success:
        frame_name = f'{frame_name_prefix}_{count}.jpg'                                     # Format frame file name
        frame_path = os.path.join(output_path, frame_name)                                  # Join frame file path
        	
        if subwindow_coords is not None:                                                    # Not None: Take subwindow image
            x1, y1, x2, y2 = subwindow_coords
            height, width, _ = image.shape     # We need the height and width of the image to prevent the coords from going out of bounds
    
            x1 = max(0, min(width, x1))
            x2 = max(0, min(width, x2))
            y1 = max(0, min(height, y1))
            y2 = max(0, min(height, y2))
            subwindow = image[y1:y2, x1:x2]
            
            cv2.imwrite(frame_path, subwindow)
        else:                                                                                # None: Take full image
            cv2.imwrite(frame_path, image)
                                                    
        success, image = vid_cap.read()
        count += 1

    vid_cap.release()

In [5]:
def extract_frames_from_input(input_data, output_folder, single_folder, max_videos=None):    
    # 1. Try distinguishing the input type
    video_paths = []

    if isinstance(input_data, list):                                            # Input is a list of file paths
        for input_item in input_data:
            # 1. Create the absolute filepath for each file
            file_path = os.path.join(input_folder_path, input_item)

            # 2. Check if input file exist and whether it has the correct extension
            if os.path.isfile(file_path) and is_valid_video_file(file_path):
                video_paths.append(file_path)

    elif isinstance(input_data, str):                                           # Input is the entire input folder
        folder_path = input_data
        file_list = os.listdir(folder_path)
        video_paths = [os.path.join(folder_path, file) for file in file_list if is_valid_video_file(file)]

    # 2. Extract clips from each video listed in video_paths
    for video_path in video_paths:
        # 2.1 File format check
        if not is_valid_video_file(video_path):
            print(f"Invalid file format: {video_path}")
            continue
        
        # 2.2 Prepare frame prefix to name the output files
        frame_prefix = "frame"
        if single_folder:
            output_path = output_folder
            frame_prefix = os.path.splitext(os.path.basename(video_path))[0]
        else:
            video_name = os.path.splitext(os.path.basename(video_path))[0]
            output_path = os.path.join(output_folder, video_name)

        # 2.3 Extract frame and save it in output_path
        extract_frames(video_path, output_path, frame_prefix, subwindow_coords)

In [6]:
if full_folder:
    # Extracting clips from all videos within the folder
    print("Directory of videos to extract frames from: ", input_folder_path)
    extract_frames_from_input(input_folder_path, output_folder_path, single_folder)

else:
    print("Extracting video clips from list of videos")
    extract_frames_from_input(file_names, output_folder_path, single_folder)

print("Frames extraction completed.")

Directory of videos to extract frames from:  /app/Videos
Frames extraction completed.
